[Numba Tutorial](https://github.com/ContinuumIO/numbapro-examples/blob/master/multigpu/multigpu_mt.py)

## Numba Newtown eval

In [1]:
import threading
import numpy as np

from math import ceil
from numba import cuda

import minterpy as mp

In [2]:
print('System has %d CUDA devices' % len(cuda.list_devices()))

System has 4 CUDA devices


In [3]:
signature = 'void(float64[:], float64[:], float64[:], float64[:], float64[:])'

In [4]:
@cuda.jit(device=True)
def monomial(
    x_single: np.ndarray,
    exponent_single: np.ndarray,
    points: np.ndarray,
):
    m = len(exponent_single)
    tmp = 1.0
    for i in range(m):
        for j in range(exponent_single[i]):
            p_ij = points[j, i]
            tmp *= x_single[i] - p_ij

    return tmp

In [5]:
def kernel(
    xx: np.ndarray,
    exponents: np.ndarray,
    points: np.ndarray,
    coefficients: np.ndarray,
    out: np.ndarray,
) -> None:
    """Evaluate the Newton polynomial multiple query points (CUDA kernel)."""
    row = cuda.grid(1)
    N = exponents.shape[0]
    if row < len(out):
        tmp = 0.0
        x_single = xx[row]
        for i in range(N):
            newton_monomials = monomial(x_single, exponents[i], points)
            tmp += newton_monomials * coefficients[i]

        out[row] = tmp

In [6]:
compiler_lock = threading.Lock()

In [7]:
def device_controller(cid, xx, exponents, points, coefficients, global_out):
    cuda.select_device(cid)                    # bind device to thread
    device = cuda.get_current_device()         # get current device
    num_devices = len(cuda.list_devices())
    
    # print some information about the CUDA card
    #prefix = '[%s]' % device
    #print(prefix, 'device_controller', cid, '| CC', device.COMPUTE_CAPABILITY)

    max_thread = device.MAX_THREADS_PER_BLOCK
    
    with compiler_lock:                        # lock the compiler
        # prepare function for this thread
        # the jitted CUDA kernel is loaded into the current context
        cuda_kernel = cuda.jit(kernel)

    # prepare data
    N = int(ceil(float(xx.shape[0]) / num_devices))
    start = int(cid * N)
    stop = None
    if cid < num_devices - 1:
        stop=int((cid+1) * N)

    # determine number of threads and blocks
    if N >= max_thread:
        ngrid = int(ceil(float(N) / max_thread))
        nthread = max_thread
    else:
        ngrid = 1
        nthread = N
        
    #print(prefix, 'grid x thread = %d x %d' % (ngrid, nthread))

    # real CUDA work
    d_xx = cuda.to_device(xx[start:stop, :])
    d_exponents = cuda.to_device(exponents)
    d_points = cuda.to_device(points)
    d_coefficients = cuda.to_device(coefficients)
    d_out = cuda.device_array((N))
    
    cuda_kernel[ngrid, nthread](
        d_xx,
        d_exponents,
        d_points,
        d_coefficients,
        d_out
    )
    
    global_out[start:stop] = d_out.copy_to_host()


In [8]:
spatial_dimension = 4
mi = mp.MultiIndexSet.from_degree(spatial_dimension, 45, 2.0)
xx = -1 + 2 * np.random.rand(1000000, spatial_dimension)
coefficients = np.random.rand(len(mi))
exponents = mi.exponents
grd = mp.Grid(mi)
points = grd.generating_points
out = np.zeros(1000000)

In [9]:
children = []
for cid, dev in enumerate(cuda.list_devices()):
    t = threading.Thread(target=device_controller, args=(cid, xx, exponents, points, coefficients, out))
    t.start()
    children.append(t)

for t in children:
    t.join()

In [ ]:
out